In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('data.xlsx')

In [3]:
df

,Country,Year,Emission,Energy efficiency,Fossil fuels,Hydrogen and fuel cells,Nuclear,Other power and storage technologies,Renewables
0,Australia,2000,274.33,0.000,0.000,0.000,0.000,0.000,0.000
1,Australia,2001,281.47,17.152,118.584,0.000,5.329,10.331,19.396
2,Australia,2002,285.92,0.000,0.000,0.000,0.000,0.000,0.000
3,Australia,2003,289.42,15.887,97.503,0.000,1.580,7.360,18.977
4,Australia,2004,300.42,55.633,61.956,0.343,35.143,11.036,44.505
...,...,...,...,...,...,...,...,...,...
513,United States,2016,4241.08,1868.364,597.518,121.460,1029.452,205.570,866.341
514,United States,2017,4196.40,1974.924,576.251,119.254,1053.926,240.804,794.754
515,United States,2018,4266.79,2464.632,678.637,132.597,1244.294,147.265,870.584
516,United States,2019,4222.79,2549.316,666.958,135.924,1488.371,394.698,1051.688


In [4]:
en = pd.read_excel('en_use.xlsx')

In [5]:
en = en.melt(id_vars = 'Country', var_name = 'Year', value_name = 'Energy use')

In [6]:
en

,Country,Year,Energy use
0,Australia,2000,2545.70
1,Austria,2000,920.70
2,Belgium,2000,NaN
3,Canada,2000,6900.56
4,Czech Republic,2000,1048.29
...,...,...,...
589,United Kingdom,2021,NaN
590,United States,2021,NaN
591,IEA,2021,122842.49
592,Brazil,2021,NaN


In [7]:
df = pd.merge_ordered(df, en)

In [8]:
df.dropna(inplace=True, ignore_index = True)
df

,Country,Year,Emission,Energy efficiency,Fossil fuels,Hydrogen and fuel cells,Nuclear,Other power and storage technologies,Renewables,Energy use
0,Australia,2000,274.33,0.000,0.000,0.000,0.000,0.000,0.000,2545.70
1,Australia,2001,281.47,17.152,118.584,0.000,5.329,10.331,19.396,2554.74
2,Australia,2002,285.92,0.000,0.000,0.000,0.000,0.000,0.000,2553.62
3,Australia,2003,289.42,15.887,97.503,0.000,1.580,7.360,18.977,2654.80
4,Australia,2004,300.42,55.633,61.956,0.343,35.143,11.036,44.505,2722.53
...,...,...,...,...,...,...,...,...,...,...
513,United States,2016,4241.08,1868.364,597.518,121.460,1029.452,205.570,866.341,54460.04
514,United States,2017,4196.40,1974.924,576.251,119.254,1053.926,240.804,794.754,54939.22
515,United States,2018,4266.79,2464.632,678.637,132.597,1244.294,147.265,870.584,56461.94
516,United States,2019,4222.79,2549.316,666.958,135.924,1488.371,394.698,1051.688,57239.83


In [9]:
df = df[['Country', 'Year', 'Emission', 'Energy use', 'Energy efficiency', 'Fossil fuels', 'Hydrogen and fuel cells', 'Nuclear', 'Other power and storage technologies', 'Renewables']]

In [10]:
df = df.set_index(["Country", "Year"])

In [11]:
exog_vars = df.columns.tolist()
exog_vars.pop(0)
exog_vars

['Energy use',
 'Energy efficiency',
 'Fossil fuels',
 'Hydrogen and fuel cells',
 'Nuclear',
 'Other power and storage technologies',
 'Renewables']

In [12]:
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

In [13]:
exog = sm.add_constant(df[exog_vars])

In [14]:
mod = PanelOLS(df.Emission, exog, entity_effects=True, time_effects=True)

In [15]:
fixed_res = mod.fit()

In [16]:
fixed_res

Dep. Variable:,Emission,R-squared:,0.9339
Estimator:,PanelOLS,R-squared (Between):,0.9499
No. Observations:,518,R-squared (Within):,0.9381
Date:,"Sat, Dec 02 2023",R-squared (Overall):,0.9501
Time:,08:18:00,Log-likelihood,-2665.5
Cov. Estimator:,Unadjusted,,
,,F-statistic:,937.22
Entities:,26,P-value,0.0000
Avg Obs:,19.923,Distribution:,"F(7,464)"
Min Obs:,11.000,,
Max Obs:,22.000,F-statistic (robust):,937.22


In [17]:
f = fixed_res.tolist()

AttributeError: 'PanelEffectsResults' object has no attribute 'tolist'